<a href="https://colab.research.google.com/github/CarolKogei/google-notebooks-dl/blob/main/resnet50_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pwd

'/content'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import libraries and modules

import os
import pandas as pd
import numpy as np
import seaborn as sns
#import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#loading the path of the image to a directory
def load_images_from_folder(folder,only_path = False, label = ""):
    if only_path == False:
        images = []
        file_name=[]
        for filename in os.listdir(folder):
            img = plt.imread(os.path.join(folder,filename))
            
            if img is not None:
                end=filename.find(".")
                file_name.append(file[0:end])
                images.append(img)
                
        return images, file_name
    else:
        path = []
        for filename in os.listdir(folder):
            img_path = os.path.join(folder,filename)
            if img_path is not None:
                path.append([label,img_path])
        return path

In [ ]:
# Load the paths on the images
images = []
path = "/content/drive/MyDrive/database/database/"
for subfolder in os.listdir(path):
    if "jpg" in os.listdir(path+subfolder)[0]:
        images += load_images_from_folder(path+subfolder,True,label = subfolder)
      
    else: 
        for d in os.listdir(path+subfolder):
            images += load_images_from_folder(path+subfolder+"/"+d,True,label = f)
            


In [ ]:
# Create a dataframe with the paths and the label for each insect
df1 = pd.DataFrame(images, columns = ["insect_gbif", "path_img"])

file_name=[]
for i in range(len(df1["path_img"])):
    temp=df1.path_img[i].split('/')[-1].split('.')[0]
    file_name.append(temp)
file_name

df1['file_name'] = file_name
display(df1.describe())

display(df1)

,insect_gbif,path_img,file_name
count,63364,63364,63364
unique,291,63364,63364
top,9364935,/content/drive/MyDrive/database/database/10368...,d028s0066
freq,888,1,1


,insect_gbif,path_img,file_name
0,1035167,/content/drive/MyDrive/database/database/10351...,d162s0011
1,1035167,/content/drive/MyDrive/database/database/10351...,d162s0012
2,1035167,/content/drive/MyDrive/database/database/10351...,d162s0013
3,1035167,/content/drive/MyDrive/database/database/10351...,d162s0018
4,1035167,/content/drive/MyDrive/database/database/10351...,d162s0017
...,...,...,...
63359,9581584,/content/drive/MyDrive/database/database/95815...,d135s0406
63360,9581584,/content/drive/MyDrive/database/database/95815...,d135s0407
63361,9581584,/content/drive/MyDrive/database/database/95815...,d135s0408
63362,9581584,/content/drive/MyDrive/database/database/95815...,d135s0409


In [ ]:
#grouping images into folders
folder_group_10 = df1.groupby(['insect_gbif']).size().reset_index(name = 'count')
folder_group_10

,insect_gbif,count
0,1035167,213
1,1035185,599
2,1035194,245
3,1035195,570
4,1035204,164
...,...,...
286,9444427,134
287,9479706,191
288,9491931,239
289,9533851,273


In [ ]:
#ranking top ten folders with highest number of images
folder_group = folder_group_10.sort_values('count',ascending = False).head(n=10)
folder_group

,insect_gbif,count
282,9364935,888
15,1035931,861
238,7508714,818
87,4475140,719
21,1036216,686
147,5755079,651
78,4474169,642
1,1035185,599
3,1035195,570
13,1035864,566


In [ ]:
#putting folder_group_10 into a differant directory
keys = df1['insect_gbif'].value_counts()[:10].to_dict().keys()
#print(keys)
import shutil

folder_group = '/content/drive/MyDrive/folder_group/'
os.mkdir(folder_group)
for folder in keys:
    src = path + folder
    shutil.copytree(src,folder_group +folder)

In [ ]:
pip install split-folders

In [ ]:
# Randomly splitting data into train, validate and testing datasets

import splitfolders
input_folder = '/content/drive/MyDrive/folder_group'
output = '/content/drive/MyDrive/final_dataset'

splitfolders.ratio(input_folder, output = output, seed =42, ratio=(0.8,0.1,0.1))

Copying files: 7000 files [01:24, 82.41 files/s]


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory = '/content/drive/MyDrive/final_dataset/train', target_size=(224,224),classes=[ '1035931','4474169','7508714','4475140','9364935','1035864','1036216','1035185','5755079','1035195' ],batch_size=20)
val_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory = '/content/drive/MyDrive/final_dataset/val', target_size=(224,224),classes=[ '1035931','4474169','7508714','4475140','9364935','1035864','1036216','1035185','5755079','1035195' ],batch_size=20)
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory = '/content/drive/MyDrive/final_dataset/test', target_size=(224,224),classes=[ '1035931','4474169','7508714','4475140','9364935','1035864','1036216','1035185','5755079','1035195' ],batch_size=20,shuffle=False)

Found 5595 images belonging to 10 classes.
Found 695 images belonging to 10 classes.
Found 710 images belonging to 10 classes.


# ResNet50 model

In [ ]:
# loading a resnet50 model
# loading modules
import tensorflow as tf
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
# 
resnet_model=ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

94781440/94765736 [==============================] - 1s 0us/step


*   when implementing pre-trained model, keras. We disable the top layer inorder to give us control of both the input and output.
*   Resnet also works optimally with 224 * 224 images hence the specification 

In [ ]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

* The pretrained model is computationally intensive, we therefore reduce its size by disabling some layers to save on time and computational power.

In [ ]:
# freezing all the layers except last block
#for layer in resnet_model.layers[:300]:
  #layer.trainable = False

In [ ]:
#freezing all layers
resnet_model.trainable = False

In [ ]:
# checking to see if layers were frozen
for i, layer in enumerate(resnet_model.layers):
  print(i,layer.name, "-", layer.trainable)

0 input_1 - False
1 conv1_pad - False
2 conv1_conv - False
3 conv1_bn - False
4 conv1_relu - False
5 pool1_pad - False
6 pool1_pool - False
7 conv2_block1_1_conv - False
8 conv2_block1_1_bn - False
9 conv2_block1_1_relu - False
10 conv2_block1_2_conv - False
11 conv2_block1_2_bn - False
12 conv2_block1_2_relu - False
13 conv2_block1_0_conv - False
14 conv2_block1_3_conv - False
15 conv2_block1_0_bn - False
16 conv2_block1_3_bn - False
17 conv2_block1_add - False
18 conv2_block1_out - False
19 conv2_block2_1_conv - False
20 conv2_block2_1_bn - False
21 conv2_block2_1_relu - False
22 conv2_block2_2_conv - False
23 conv2_block2_2_bn - False
24 conv2_block2_2_relu - False
25 conv2_block2_3_conv - False
26 conv2_block2_3_bn - False
27 conv2_block2_add - False
28 conv2_block2_out - False
29 conv2_block3_1_conv - False
30 conv2_block3_1_bn - False
31 conv2_block3_1_relu - False
32 conv2_block3_2_conv - False
33 conv2_block3_2_bn - False
34 conv2_block3_2_relu - False
35 conv2_block3_3_conv - 

* False means it was frozen and therefore not trainable while True means it's executable whenever the model runs

In [ ]:
# connecting the pre-trained model with our sequential model
model = tf.keras.models.Sequential()
model.add(resnet_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_1 (Dense)             (None, 10)                1003530   
                                                                 
Total params: 24,591,242
Trainable params: 1,003,530
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
# compiling the model
optimizer = Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches, validation_data=val_batches, epochs=10, verbose=2)

Epoch 1/10
36/36 - 224s - loss: 4.3556 - accuracy: 0.1535 - val_loss: 3.0727 - val_accuracy: 0.1324 - 224s/epoch - 6s/step
Epoch 2/10
36/36 - 220s - loss: 1.9175 - accuracy: 0.4113 - val_loss: 2.2896 - val_accuracy: 0.3583 - 220s/epoch - 6s/step
Epoch 3/10
36/36 - 220s - loss: 1.0798 - accuracy: 0.6085 - val_loss: 1.6984 - val_accuracy: 0.4576 - 220s/epoch - 6s/step
Epoch 4/10
36/36 - 219s - loss: 0.4831 - accuracy: 0.8592 - val_loss: 1.4162 - val_accuracy: 0.5482 - 219s/epoch - 6s/step
Epoch 5/10
36/36 - 219s - loss: 0.2232 - accuracy: 0.9577 - val_loss: 1.2831 - val_accuracy: 0.5842 - 219s/epoch - 6s/step
Epoch 6/10
36/36 - 219s - loss: 0.1252 - accuracy: 0.9915 - val_loss: 1.2242 - val_accuracy: 0.6029 - 219s/epoch - 6s/step
Epoch 7/10
36/36 - 219s - loss: 0.0897 - accuracy: 0.9986 - val_loss: 1.1752 - val_accuracy: 0.6216 - 219s/epoch - 6s/step
Epoch 8/10
36/36 - 219s - loss: 0.0662 - accuracy: 1.0000 - val_loss: 1.1479 - val_accuracy: 0.6345 - 219s/epoch - 6s/step
Epoch 9/10
36/36

* the above model ran after freezing 250 layers

* These accuracies were run after freezing all layers of the resnet50

In [ ]:
model.fit(x=train_batches, validation_data=val_batches, epochs=10, verbose=2)

Epoch 1/10
36/36 - 231s - loss: 5.0685 - accuracy: 0.1155 - val_loss: 3.4656 - val_accuracy: 0.1554 - 231s/epoch - 6s/step
Epoch 2/10
36/36 - 227s - loss: 2.5423 - accuracy: 0.2789 - val_loss: 2.3381 - val_accuracy: 0.2820 - 227s/epoch - 6s/step
Epoch 3/10
36/36 - 225s - loss: 1.5178 - accuracy: 0.5070 - val_loss: 1.9553 - val_accuracy: 0.4187 - 225s/epoch - 6s/step
Epoch 4/10
36/36 - 225s - loss: 0.9514 - accuracy: 0.6676 - val_loss: 1.5768 - val_accuracy: 0.5065 - 225s/epoch - 6s/step
Epoch 5/10
36/36 - 225s - loss: 0.6813 - accuracy: 0.7746 - val_loss: 1.2983 - val_accuracy: 0.5813 - 225s/epoch - 6s/step
Epoch 6/10
36/36 - 226s - loss: 0.4563 - accuracy: 0.8676 - val_loss: 1.2843 - val_accuracy: 0.6144 - 226s/epoch - 6s/step
Epoch 7/10
36/36 - 229s - loss: 0.3011 - accuracy: 0.9324 - val_loss: 1.0806 - val_accuracy: 0.6388 - 229s/epoch - 6s/step
Epoch 8/10
36/36 - 229s - loss: 0.2070 - accuracy: 0.9718 - val_loss: 1.0333 - val_accuracy: 0.6705 - 229s/epoch - 6s/step
Epoch 9/10
36/36

* The accuracies below 300 layers frozen

In [ ]:
# accuracy with all layers of the base model frozen
model.fit(x=train_batches, validation_data=val_batches, epochs=10, verbose=2)


Epoch 1/10
36/36 - 332s - loss: 4.5182 - accuracy: 0.1521 - val_loss: 2.8447 - val_accuracy: 0.2532 - 332s/epoch - 9s/step
Epoch 2/10
36/36 - 225s - loss: 2.3252 - accuracy: 0.3254 - val_loss: 2.2368 - val_accuracy: 0.3568 - 225s/epoch - 6s/step
Epoch 3/10
36/36 - 225s - loss: 1.5689 - accuracy: 0.4789 - val_loss: 1.7551 - val_accuracy: 0.4273 - 225s/epoch - 6s/step
Epoch 4/10
36/36 - 225s - loss: 1.0327 - accuracy: 0.6451 - val_loss: 1.4233 - val_accuracy: 0.5266 - 225s/epoch - 6s/step
Epoch 5/10
36/36 - 225s - loss: 0.6011 - accuracy: 0.7958 - val_loss: 1.2021 - val_accuracy: 0.6000 - 225s/epoch - 6s/step
Epoch 6/10
36/36 - 224s - loss: 0.3795 - accuracy: 0.8831 - val_loss: 1.1148 - val_accuracy: 0.6245 - 224s/epoch - 6s/step
Epoch 7/10
36/36 - 222s - loss: 0.2803 - accuracy: 0.9394 - val_loss: 1.0178 - val_accuracy: 0.6388 - 222s/epoch - 6s/step
Epoch 8/10
36/36 - 223s - loss: 0.2014 - accuracy: 0.9620 - val_loss: 0.9684 - val_accuracy: 0.6676 - 223s/epoch - 6s/step
Epoch 9/10
36/36